In [17]:
import mysql.connector
import getpass
#add any database name u want to ask
db_name="sample_db"
# to generate db from the sample if u have already setted a db skip this cell
try:
    password = getpass.getpass("Enter database password: ")
    with mysql.connector.connect(
        host="localhost",
        user="root",
        password=password,
        database=db_name,
        autocommit=True
    ) as conn:
        # Open the SQL script file and read its contents with explicit encoding
        with open('./sql.sql', 'r', encoding='utf-8') as f:
            sql_script = f.read()

        # Split the script into individual SQL statements
        sql_statements = sql_script.split(';')

        # Execute each SQL statement
        cursor = conn.cursor()
        for statement in sql_statements:
            if statement.strip():  # Skip empty statements
                cursor.execute(statement)

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    # Close the cursor and database connection
    if 'cursor' in locals():
        cursor.close()

    if 'conn' in locals() and conn.is_connected():
        conn.close()

Error: 1049 (42000): Unknown database 'sample_db'


In [9]:
import mysql.connector

# Connect to the MySQL database
conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database=db_name
)

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Execute a SELECT query
cursor.execute('SELECT * FROM employees')

# Fetch the query results
rows = cursor.fetchall()

# Print the results
for row in rows:
    print(row)

# Close the cursor and database connection
cursor.close()
conn.close()

(1, 'John', 'Doe', 50000, 1)
(2, 'Jane', 'Smith', 60000, 2)
(3, 'Bob', 'Johnson', 55000, 3)


In [10]:
import os

os.environ['OPENAI_API_KEY'] = "...." #generated API key

In [11]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain_openai import OpenAI

# Replace these with your MySQL connection details
mysql_host = "localhost"
mysql_user = "root"
mysql_password = ""
mysql_db = db_name

# Create a MySQL engine
db = SQLDatabase.from_uri(f"mysql+mysqlconnector://{mysql_user}:{mysql_password}@{mysql_host}/{mysql_db}")

# Install the 'openai' module if not already installed
# pip install openai

# Create an OpenAI language model
llm = OpenAI(temperature=0)

# Create a SQL agent toolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# Create a SQL agent executor
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [12]:
agent_executor.invoke("who many rows in employees table")



> Entering new AgentExecutor chain...
 I need to use sql_db_query to count the number of rows in the employees table
Action: sql_db_query
Action Input: SELECT COUNT(*) FROM employees[(3,)]3 rows in the employees table
Final Answer: 3 rows in the employees table

> Finished chain.


{'input': 'who many rows in employees table',
 'output': '3 rows in the employees table'}